In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def lstsqk(x, y):
    return np.multiply(x, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0)

In [ ]:
def lstsqkerr(x, y):
    return 1 / np.sqrt(len(x)) * np.sqrt(np.multiply(y, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0) - lstsqk(x,y) ** 2)

# First

In [ ]:
df1 = pd.read_excel('experimental_data.xlsx', sheet_name = 'First').sort_values(by = ["h, см"]).reset_index(drop = True)
df2 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Third').sort_values(by = ["h, см"]).reset_index(drop = True)

In [ ]:
# In gauissian
d1 = 3.90e-1
d2 = 5.25e-1
sigma_d1 = 0.05e-1
L1 = 50
L2 = 50
sina = 0.2
g = 981
sigma_t = 0.5

# General Data

In [ ]:
df1['dP, бария'] = df1['h, см'] * g * sina
tdf = pd.DataFrame([df1['t_1, с'], df1['t_2, с'], df1['t_3, с']])
df1['Q, см^3/с'] = df1['V, см^3'] / tdf.mean()
df1['sigma_Q, см^3/с'] = df1['Q, см^3/с'] * np.sqrt(tdf.std() ** 2 + sigma_t ** 2) / tdf.mean()

In [ ]:
df2['dP, бария'] = df2['h, см'] * g * sina
tdf = pd.DataFrame([df2['t_1, с'], df2['t_2, с'], df2['t_3, с']])
df2['Q, см^3/с'] = df2['V, см^3'] / tdf.mean()
df2['sigma_Q, см^3/с'] = df2['Q, см^3/с'] * np.sqrt(tdf.std() ** 2 + sigma_t ** 2) / tdf.mean()

In [ ]:
plot_label = ["$L = {:.2f}$ см, $d = {:.2f} \pm {:.2f}$ мм".format(L1, d1 * 10, sigma_d1 * 10), "$L = {:.2f}$ см, $d = {:.2f} \pm {:.2f}$ мм".format(L2, d2 * 10, sigma_d1 * 10)]

# Plot raw data

In [ ]:
def plot(ax, fig, df, i):
    # Show grid
    ax.grid(visible = True, linestyle = '--', which='both')
    
    # Set axis format to scientific
    fig.canvas.draw()
    plt.ticklabel_format(axis='both', style='sci', scilimits=(0,0))

    plt.xlabel('$\Delta{P}$, бария')
    plt.ylabel('$Q$, $см^3 / с$')

    ax.scatter(df['dP, бария'], df['Q, см^3/с'], label = plot_label[i - 1])

    # Set axis limits
    ax.set_xlim(left = 0)
    ax.set_ylim(bottom = 0)

    # Set title
    ax.set_title('Рис. {}. Зависимость $Q$ от $\Delta P$'.format(i))

    # Show plot
    plt.legend()
    plt.show()

    fig.savefig('plots/plot{}.pdf'.format(i))

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))

ax.xaxis.set_major_locator(plt.MultipleLocator(200))
ax.yaxis.set_major_locator(plt.MultipleLocator(20))
ax.xaxis.set_minor_locator(plt.MultipleLocator(100))
ax.yaxis.set_minor_locator(plt.MultipleLocator(5))
        
plot(ax, fig, df1, 1)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))

ax.xaxis.set_major_locator(plt.MultipleLocator(100))
ax.yaxis.set_major_locator(plt.MultipleLocator(20))
ax.xaxis.set_minor_locator(plt.MultipleLocator(20))
ax.yaxis.set_minor_locator(plt.MultipleLocator(5))
        
plot(ax, fig, df2, 2)

# Plot laminar part of flow

In [ ]:
def plot_lin(ax, fig, df, p, err, i):
    # Show grid
    ax.grid(visible = True, linestyle = '--', which='both')
    
    # Set axis format to scientific
    fig.canvas.draw()
    plt.ticklabel_format(axis='both', style='sci', scilimits=(0,0))

    plt.xlabel('$\Delta{P}$, бария')
    plt.ylabel('$Q$, $см^3 / с$')
    
    x = np.linspace(0, 11 / 10 * max(df['dP, бария']), 1000)
    
    ax.plot(x, p[0] * x + p[1], label = 'Линейная модель: $Q = a + b \cdot \Delta P$')
    ax.plot([], [], ' ', label = '$k = {k:.3f} \pm {dk:.3f}, C = {C:.3f} \pm {dC:.3f}$'.format(k = p[0], dk = err[0], C = p[1], dC = err[1]))
    
    ax.scatter(df['dP, бария'], df['Q, см^3/с'])
    # Set axis limits
    ax.set_xlim(left = 0)
    ax.set_ylim(bottom = 0)

    # Set title
    ax.set_title('Рис. {}. Зависимость $Q$ от $\Delta P$ (Ламинарный поток)'.format(i))

    # Show plot
    plt.legend()
    plt.show()

    fig.savefig('plots/plot{}.pdf'.format(i))

# Use least squares to fit a linear model

In [ ]:
lamdf1 = df1.iloc[0:11]
lamdf2 = df2
p1, v1 = np.polyfit(lamdf1['dP, бария'], lamdf1['Q, см^3/с'], deg = 1, cov = True)
err1 = np.sqrt(np.diag(v1))
p2, v2 = np.polyfit(lamdf2['dP, бария'], lamdf2['Q, см^3/с'], deg = 1, cov = True)
err2 = np.sqrt(np.diag(v1))

# Analytical least squares

In [ ]:
dferr1 = pd.DataFrame()
dferr1['Q, см^3/с'] = lamdf1['Q, см^3/с']
dferr1['Q^2, (см^3/c)^2'] = lamdf1['Q, см^3/с'] * lamdf1['Q, см^3/с']
dferr1['dP, бария'] = lamdf1['dP, бария']
dferr1['dP^2, бария^2'] = lamdf1['dP, бария'] * lamdf1['dP, бария']
dferr1['Q * dP, бария * см^3/с'] = lamdf1['dP, бария'] * lamdf1['Q, см^3/с']

In [ ]:
dferr2 = pd.DataFrame()
dferr2['Q, см^3/с'] = lamdf2['Q, см^3/с']
dferr2['Q^2, (см^3/c)^2'] = lamdf2['Q, см^3/с'] * lamdf2['Q, см^3/с']
dferr2['dP, бария'] = lamdf2['dP, бария']
dferr2['dP^2, бария^2'] = lamdf2['dP, бария'] * lamdf2['dP, бария']
dferr2['Q * dP, бария * см^3/с'] = lamdf2['dP, бария'] * lamdf2['Q, см^3/с']

In [ ]:
dferr1.to_excel('output/dfmnk1.xlsx')
dferr2.to_excel('output/dfmnk2.xlsx')

# More plots

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))

ax.xaxis.set_major_locator(plt.MultipleLocator(200))
ax.yaxis.set_major_locator(plt.MultipleLocator(20))
ax.xaxis.set_minor_locator(plt.MultipleLocator(100))
ax.yaxis.set_minor_locator(plt.MultipleLocator(5))
        
plot_lin(ax, fig, lamdf1, p1, err1, 3)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))

ax.xaxis.set_major_locator(plt.MultipleLocator(100))
ax.yaxis.set_major_locator(plt.MultipleLocator(20))
ax.xaxis.set_minor_locator(plt.MultipleLocator(20))
ax.yaxis.set_minor_locator(plt.MultipleLocator(5))
        
plot_lin(ax, fig, lamdf2, p2, err2, 4)

In [ ]:
mu_1 = np.pi * d1 ** 4 / (128 * p1[0] * L1)

In [ ]:
sigma_mu_1 = mu_1 * np.sqrt((4 * sigma_d1 / d1) ** 2 + (err1[0] / p1[0]) ** 2)

In [ ]:
mu_2 = np.pi * d2 ** 4 / (128 * p2[0] * L2)

In [ ]:
sigma_mu_2 = mu_2 * np.sqrt((4 * sigma_d1 / d2) ** 2 + (err2[0] / p2[0]) ** 2)

# Reynolds number

In [ ]:
rho = 1.184e-3 # Density at 25C
Q_critical = 63

In [ ]:
Re = rho * Q_critical / (np.pi * d1 / 2 * mu_1)

In [ ]:
df3 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Second').sort_values(by = ["L, см"]).reset_index(drop = True)

In [ ]:
df3['dP, бария'] = df3['h, см'] * g * sina

In [ ]:
V = 5e3 # cm^3
t = 107 # sec
Q = V / t
l = 0.2 * rho * Q / (np.pi * mu_1)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))

ax.xaxis.set_major_locator(plt.MultipleLocator(10))
ax.yaxis.set_major_locator(plt.MultipleLocator(200))
ax.xaxis.set_minor_locator(plt.MultipleLocator(2))
ax.yaxis.set_minor_locator(plt.MultipleLocator(40))

# Show grid
ax.grid(visible = True, linestyle = '--', which='both')

# Set axis format to scientific
fig.canvas.draw()
plt.ticklabel_format(axis='both', style='sci', scilimits=(0,0))

plt.xlabel('$L$, см')
plt.ylabel('$\Delta P$, бария')

ax.scatter(df3['L, см'], df3['dP, бария'])

# Set axis limits
ax.set_xlim(left = 0)
ax.set_ylim(bottom = 0)

# Set title
ax.set_title('Рис. 5. Зависимость $\Delta P$ от $L$')

# Show plot
plt.legend()
plt.show()

fig.savefig('plots/plot5.pdf')